In [5]:
import pandas as pd
import pyodbc

server = 'DESKTOP-1Q56108' # Nome do servidor SQL Server
database = 'Python'        # Nome do banco de dados
conexaoDB = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'f'SERVER={server};'f'DATABASE={database};''Trusted_Connection=yes;')
cursor = conexaoDB.cursor() # criando o cursor de comando para executar as consultas (queries)

#### Importação dos dataframes

In [6]:
Produtos = pd.read_excel(r'C:\Users\Cliente\Desktop\Engenharia_de_dados_com_Python\SGBDS_PYTHON\4_ETL Pandas\arquivos_excel\Produto.xlsx')
Categoria = pd.read_excel(r'C:\Users\Cliente\Desktop\Engenharia_de_dados_com_Python\SGBDS_PYTHON\4_ETL Pandas\arquivos_excel\Categoria.xlsx')
Itens = pd.read_excel(r'C:\Users\Cliente\Desktop\Engenharia_de_dados_com_Python\SGBDS_PYTHON\4_ETL Pandas\arquivos_excel\items.xlsx')
Ordens = pd.read_excel(r'C:\Users\Cliente\Desktop\Engenharia_de_dados_com_Python\SGBDS_PYTHON\4_ETL Pandas\arquivos_excel\Ordens.xlsx')
Clientes = pd.read_csv(r'C:\Users\Cliente\Desktop\Engenharia_de_dados_com_Python\SGBDS_PYTHON\4_ETL Pandas\arquivos_csv\Clientes.csv')

#### Deletando os dados de todas as tabelas

In [7]:
cursor.execute("""
TRUNCATE TABLE [dbo].[Categoria];
TRUNCATE TABLE [dbo].[Ordens];
TRUNCATE TABLE [dbo].[Itens];
TRUNCATE TABLE [dbo].[Produtos];
TRUNCATE TABLE [dbo].[Clientes];
""")
cursor.commit() # Validar os dados do SQL Server para não bloquear o usuário

#### Inserção dos dados

In [8]:
Produtos.head(3)

,ID,Name,Price,Id_Category
0,0,Sapato vermelho,96,0
1,1,Bolsa mais que Velha,8000,0
2,2,Calça,75,0


In [9]:
for index, linha in Produtos.iterrows():
    cursor.execute("INSERT INTO [Produtos](id, nome, preco, id_categoria) values (?, ?, ?, ?)",
                    linha.ID, linha.Name, linha.Price, linha.Id_Category)

cursor.commit()

In [10]:
for index, linha in Categoria.iterrows():
    # Insere os dados nas colunas
    cursor.execute("INSERT INTO [Categoria](ID, categoria) values(?, ?)", linha.id, linha.nome)

cursor.commit()   

In [11]:
Itens['total_price'] = Itens['total_price'].astype(float)

for index, linha in Itens.iterrows():
    # Insere os dados nas colunas
    cursor.execute("INSERT INTO [Itens](id, order_id, product_id, quantity, total_price) values(?, ?, ?, ?, ?)",
                   linha.id, linha.order_id, linha.product_id, linha.quantity, linha.total_price)

cursor.commit()  

In [12]:
Ordens['created_at'] = pd.to_datetime(Ordens['created_at'])

for index, linha in Ordens.iterrows():
    # Insere os dados nas colunas
    cursor.execute("INSERT INTO [Ordens](id, created_at, customer_id, situacao) values(?, ?, ?, ?)",
                   linha.id, linha.created_at, linha.customer_id, linha.status)
cursor.commit()


In [ ]:
Clientes['created_at'] = pd.to_datetime(Clientes['created_at'])
Clientes['email'] = Clientes['email'].fillna('Sem registro')
Clientes['street'] = Clientes['street'].fillna('Sem info')
Clientes['number'] = Clientes['number'].fillna('Sem número')
Clientes['additionals'] = Clientes['additionals'].fillna('Sem info')

for index, linha in Clientes.iterrows():
    # Converter para o tipo str antes da inserção. É necessário fazer isso por conta dos caracteres especiais, 
    # No SQL Server é um pouco mais chato quando há caracteres especiais
    linha.email = str(linha.email)
    linha.state = str(linha.state)
    linha.street = str(linha.street)
    linha.number = str(linha.number)
    linha.additionals = str(linha.additionals)

    # Insere os dados nas colunas
    cursor.execute("INSERT INTO [Clientes](id, created_at, first_name, last_name, email, cell_phone, country, state_brazil,"
                   "street, number, additionals)values(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                   linha.id, linha.created_at, linha.first_name, linha.last_name, linha.email, linha.cell_phone,
                   linha.country, linha.state, linha.street, linha.number, linha.additionals)

cursor.commit()

In [ ]:
# Fecha o cursor e a minha conexão com o banco de dados
cursor.close()
conexaoDB.close()